In [72]:
import numpy as np
import pandas as pd
import os,sys

In [73]:
path_data = os.path.join(os.getcwd(),os.pardir,os.pardir,'data')
data_list = os.listdir(path_data)

In [74]:
year = 2012
filename = 'mmsi-daily-csvs-10-v2-' + str(year)
fishing_data_path = os.path.join(path_data,filename)
# encoding='ISO-8859-1' for read_csv

In [75]:
import pandas as pd
from datetime import datetime, timedelta

# Define the bin size for latitude and longitude
lat_bin_size = 5
lon_bin_size = 5

def calculate_bin(value, bin_size):
    bin_center = ((value + bin_size/2) // bin_size) * bin_size
    return bin_center

start_date = datetime(year, 1, 1)
end_date = datetime(year, 12, 31)
delta = timedelta(days=1)

lat_min, lat_max = -80, 80
lon_min, lon_max = -180, 180

lat_bins = np.arange(lat_min, lat_max + lat_bin_size, lat_bin_size)
lon_bins = np.arange(lon_min, lon_max + lon_bin_size, lon_bin_size)

# Initialize the aggregated data
aggregated_data = pd.DataFrame()

current_date = start_date
while current_date <= end_date:
    file_name = current_date.strftime('%Y-%m-%d') + '.csv'
    file_path = os.path.join(fishing_data_path, file_name)
    
    if os.path.exists(file_path):
        daily_data = pd.read_csv(file_path)
        daily_data = daily_data[['cell_ll_lat', 'cell_ll_lon', 'fishing_hours']]
        daily_data = daily_data[daily_data['fishing_hours'] > 0]

    daily_data['lat_bin'] = pd.cut(daily_data['cell_ll_lat'], bins = lat_bins)
    daily_data['lon_bin'] = pd.cut(daily_data['cell_ll_lon'], bins = lon_bins)
    daily_grouped = daily_data.groupby(['lat_bin', 'lon_bin']).agg({'fishing_hours': 'mean'}).reset_index()

    aggregated_data = pd.concat([aggregated_data, daily_grouped])
    current_date += delta

# Final aggregation across all days, summing up 'fishing_hours' for each bin
aggregated_data = aggregated_data.groupby(['lat_bin', 'lon_bin']).sum().reset_index()

In [76]:
# import pandas as pd
# from datetime import datetime, timedelta

# # Define the bin size for latitude and longitude
# lat_bin_size = 5
# lon_bin_size = 5

# def calculate_bin(value, bin_size):
#     bin_center = ((value + bin_size/2) // bin_size) * bin_size
#     return bin_center

# start_date = datetime(year, 1, 1)
# end_date = datetime(year, 12, 31)
# delta = timedelta(days=1)

# # Initialize the aggregated data
# aggregated_data = pd.DataFrame()

# current_date = start_date
# while current_date <= end_date:
#     file_name = current_date.strftime('%Y-%m-%d') + '.csv'
#     file_path = os.path.join(fishing_data_path, file_name)
    
#     if os.path.exists(file_path):
#         daily_data = pd.read_csv(file_path)
#         daily_data = daily_data[['cell_ll_lat', 'cell_ll_lon', 'fishing_hours']]
#         daily_data = daily_data[daily_data['fishing_hours'] > 0]

#     daily_data['lat_bin'] = daily_data['cell_ll_lat'].apply(calculate_bin, bin_size=lat_bin_size)
#     daily_data['lon_bin'] = daily_data['cell_ll_lon'].apply(calculate_bin, bin_size=lon_bin_size)
    
#     # Group by the binned lat/lon and sum the 'fishing_hours', then compute the mean
#     daily_grouped = daily_data.groupby(['lat_bin', 'lon_bin']).agg({'fishing_hours': 'mean'}).reset_index()
    
#     # Concatenate the grouped data to the aggregated data
#     aggregated_data = pd.concat([aggregated_data, daily_grouped])
    
#     # Increment the date by one day
#     current_date += delta

# # Final aggregation across all days, summing up 'fishing_hours' for each bin
# aggregated_data = aggregated_data.groupby(['lat_bin', 'lon_bin']).sum().reset_index()


In [77]:
save_folder = os.path.join(os.getcwd(),os.pardir,os.pardir,'data','bin_fishing_activity')
if not os.path.exists(save_folder):
    os.makedirs(save_folder)

save_file = os.path.join(save_folder,'bin_fishing_activity_' + str(year) + '.csv')
aggregated_data.to_csv(save_file, index=False)